# <center>Capstone Projet<center>

## 1.Introduction 

**Set up a delivery service base for delivering most popular restaurants' food, in Manhattan, New York, USA.**

Since the outbreak of COVID, most people work from home and have more possiblity ordering foods from their favorite restaurants in the neighborhoods.   
There are more and more orders to take, for restaurants, that's a good news.  
But for those who deliver food, their works are getting harder, so they need a place to go back and take a break after the delivery.  
In order to improve the quality of the delivery and to optimize the itinerary, perhaps the delivery service provider wants to know where is the good place to set a base for delivery guys to take a break and also to be close to the popular restaurants in the neighborhoods. 

## 2.Data

Scraping Neighborhoods in New York City Table from Wikipedia and extrat the neighborhoods in Manhattan.  
https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City   

Get Coordinates of neighborhoods from Geopy Client.

Chose the top 5 neighborhoods who have more habitat in those areas.

Use Foursquare Location Data to get the popular spots inforamtion, then cluster to find the good aera to set up a base.

## 3.Methodology 

## 4.Results

## 5.Discussion

## 6.Conclusion 